### In this homework assigment, you are required to apply the neural fitted Q-iteration algorithm to a pre-collected dataset for batch (offline) policy optimisation. Please follow the instructions detailed below.

Step 1: Generate an offline dataset. Consider the CartPole example. We will use a sub-optimal policy for data generation. Specifically, consider the following deterministic policy $\pi_b$ that returns 0 (left) if the pole angle is negative and 1 otherwise. To allow exploration, we set the behavior policy to be a mixture of $\pi_b$ and a uniform random policy. Specifically, the agent will follow the uniform random policy or $\pi_b$ with equal probability. We simulate 100 episodes under this policy. This yields the offline dataset.

Step 2: Fitted Q-iteration. We will apply the neural fitted Q-iteration (FQI) algorithm to this offline data to compute an optimal policy with three different choices of $\gamma$, corresponding to 0.95, 0.99 and 1. Please refer to Page 43 of Lecture 5 for the pseudocode of FQI in batch settings. We repeat the Q-iteration 20 times, e.g., apply supervised learning algorithms 20 times to learn the optimal Q-function. The initial Q-estimator can be set to a zero function. Each iteration yields a Q-estimator, based on which we can derive an estimated optimal policy. In total, we obtain 20 $\times$ 3 (3 choices of $\gamma$) different policies.

* To combine FQI with neural networks, we consider using the [MLPregressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) function. We can use the default neural network architecture (no need to specify no. of layers or no. of hidden nodes per layer). We may set the maximum number of iterations to 500.

* In this example, we only have two actions (either pushing the cart to the left or to the right). As such, it would be better to use the second type of value function approximators on Page 11 of Lecture 5 (e.g., for each action, use a separate model for the value). The last type of approximators would be preferred in settings where we have a large action space.

* The TD target depends on whether the current state is a terminal state or not. For a nonterminal state, the TD target is constructed as in the lecture slide. For a terminal state, the TD target is equal to the immediate reward.

Step 3: Policy evaluation. For each of the computed 60 policies, we use the Monte Carlo method to evaluate the expected return under this policy, by generating 1000 episodes. Finally, plot all the returns in a single figure and comment on the results.

In [1]:
# Neccesary packages for the implementation
from sklearn.neural_network import MLPRegressor
import gymnasium as gym
import numpy as np

# Imports enviroment CartPole from Gymnasium
env = gym.make('CartPole-v1')

From seminar code, some description of the environment:

#### Action
Two discrete actions: a=0 indicates pushing cart to the left and a=1 pushes the cart to the right.  The amount the velocity reduced or increased does not only depends on the direction you are moving but also on the angle the pole is pointing. 

#### Reward
 Reward is +1 for every step taken, including the termination step.
 
#### Observation
There are 4 observations returned by the environment after each action taken by an agent:
- Cart position:  a number between `-4.8` and `4.8`
- Cart velocity: a number between `-inf`and `inf`
- Pole angle: an angle between -24&deg; and 24&deg; 
-  Pole velocity at tip: a number between `-inf`and `inf`

#### Termination
- Cart position is smaller or greater than `-2.4` or `2.4`
- Pole Angle is smaller or greater than -12&deg; or 12&deg;
- Episode length is longer than 200

### Step 1
Generate an offline dataset. Consider the CartPole example. We will use a sub-optimal policy for data generation. Specifically, consider the following deterministic policy $\pi_b$ that returns 0 (left) if the pole angle is negative and 1 otherwise. To allow exploration, we set the behavior policy to be a mixture of $\pi_b$ and a uniform random policy. Specifically, the agent will follow the uniform random policy or $\pi_b$ with equal probability. We simulate 100 episodes under this policy. This yields the offline dataset.

In [2]:
# Policy \pi_b
def action(angle):
    if angle < 0:
        return 0
    else:
        return 1

In [3]:
# D is where I will save all the observations for the training of the NNs (offline batch)
# Each key represent an episode. 
# Saves observations  of the environment in the form (S_t, a_t, S_{t+1}, done)
# It is not neccesary to save the reward since this is always 1
def gen_data(D, episodes):
    for i in range(episodes):
        D[i] = []
        S = env.reset()[0]
        a = action(S[2])
        done = False
        while not done:
            # Takes a step
            next_S, r, done, info, _ = env.step(a)
            
            # Saves observations  of the environment in the form (S_t, a_t, S_{t+1}, done)
            D[i].append([S, a, next_S, done])
            
            # Toss a coin: follow deterministic policy \pi_b or random action is selected
            if np.random.binomial(1, p=0.5) == 1:
                # Action from policy \pi_b
                a = action(next_S[2])
            else:
                # Random action
                a = env.action_space.sample()
            S = next_S
    return D

In [4]:
episodes = 100
D = {}
D = gen_data(D, episodes)

### Step 2
Fitted Q-iteration. We will apply the neural fitted Q-iteration (FQI) algorithm to this offline data to compute an optimal policy with three different choices of $\gamma$, corresponding to 0.95, 0.99 and 1. Please refer to Page 43 of Lecture 5 for the pseudocode of FQI in batch settings. We repeat the Q-iteration 20 times, e.g., apply supervised learning algorithms 20 times to learn the optimal Q-function. The initial Q-estimator can be set to a zero function. Each iteration yields a Q-estimator, based on which we can derive an estimated optimal policy. In total, we obtain 20 $\times$ 3 (3 choices of $\gamma$) different policies.

* To combine FQI with neural networks, we consider using the [MLPregressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) function. We can use the default neural network architecture (no need to specify no. of layers or no. of hidden nodes per layer). We may set the maximum number of iterations to 500.

* In this example, we only have two actions (either pushing the cart to the left or to the right). As such, it would be better to use the second type of value function approximators on Page 11 of Lecture 5 (e.g., for each action, use a separate model for the value). The last type of approximators would be preferred in settings where we have a large action space.

* The TD target depends on whether the current state is a terminal state or not. For a nonterminal state, the TD target is constructed as in the lecture slide. For a terminal state, the TD target is equal to the immediate reward.


#### Initialisation

In [5]:
def initialisation():
    # This is the input of the neural network of the form (S_t, a_t)
    # Recall the S_t is conformed by physical states (velocities and angles)
    input0 = []
    input1 = []

    # Tuples of the next state
    next_S0 = []
    next_S1 = []
    S = []

    # Target
    y0 = []
    y1 = []

    # Wheter state is terminal or not
    done0 = []
    done1 = []

    for ep in range(episodes):
        for step in range(len(D[ep])):
            # Saves tuple for next state
            S.append(D[ep][step][0])

            # If action is 1, saves all the training input and output for this action
            if D[ep][step][1] == 1:
                input1.append(D[ep][step][0])
                y1.append(1)
                next_S1.append(D[ep][step][2])
                done1.append(D[ep][step][3])

            # If action is 0, saves all the training input and output for this action
            else:
                input0.append(D[ep][step][0])
                y0.append(1)
                next_S0.append(D[ep][step][2])
                done0.append(D[ep][step][3])

    # Training of 2 NNs
    regr0 = MLPRegressor(random_state=1, max_iter=500).fit(input0, y0)
    regr1 = MLPRegressor(random_state=1, max_iter=500).fit(input1, y1)

    # Prediction using 2 NNs just trained
    Q0_hat = regr0.predict(next_S0)
    Q1_hat = regr1.predict(next_S1)

    return input0, input1, Q0_hat, Q1_hat, next_S0, next_S1, S, done0, done1

In [6]:
# Calculates the TD target 
def calc_target(gamma, Q_hat, done):
    y = []
    for i in range(len(Q_hat)):
        # If terminal state, target is 1
        if done[i]:
            y.append(1)
        # If non terminal state, target computed as in lecture slides
        else:
            y.append(1 + gamma * Q_hat[i])
    return y

# Computes the maximum between the two Qfunction estimators
def compute_max(Q0_final, Q1_final, S):
    Q = {}
    for i in range(len(Q0_final)):
        if Q0_final[i] > Q1_final[i]:
            Q[tuple(S[i]), 0] = Q0_final[i]
        else:
            Q[tuple(S[i]), 1] = Q1_final[i]
    return Q

In [7]:
gammas = [0.95, 0.99, 1]
N_Qiterations = 2
k = 0
gamma = 0.95


def NFQ_it(gamma, N_Qiterations):
    input0, input1, Q0_hat, Q1_hat, next_S0, next_S1, S, done0, done1 = initialisation()
    Q = {}
    for i in range(N_Qiterations):

        if (1+i)%5 == 0:
            print(i+1)
            #print(Q[i-1])

        Q0_est = Q0_hat 
        Q1_est = Q1_hat

        # If terminal state, target should be 1
        y0 = calc_target(gamma, Q0_est, done0)
        y1 = calc_target(gamma, Q1_est, done1)

        # Trainig of the NNs with TD target and input are states
        regr0 = MLPRegressor(random_state=1, max_iter=500).fit(input0, y0)
        regr1 = MLPRegressor(random_state=1, max_iter=500).fit(input1, y1)

        # Prediction of Q_value functions for each action (0, 1)
        Q0_hat = regr0.predict(next_S0)
        Q1_hat = regr1.predict(next_S1)
        
        
        # To compute the optimal policy, I evaluate both NNs for the state vector S and retrieve the max
        Q0_final = regr0.predict(S)
        Q1_final = regr1.predict(S)
        
        Q[i + 1] = compute_max(Q0_final, Q1_final, S)
        
    return Q, S
Q = {}
for gamma in gammas:
    Q[gamma], S = NFQ_it(gamma, N_Qiterations)


In [8]:
Q[0.95][1][tuple(S[0]), 1]

1.9504562050302854